# Tools and Agents

## Using in-built tools

In [1]:
from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper


# Create wikipedia tool
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)
print(wiki_tool.name)

# Create arxiv tool
arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
print(arxiv_tool.name)

wikipedia
arxiv


## Creating custom tools

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Creating retriever
docs = WebBaseLoader('https://python.langchain.com/docs/integrations/tools/').load()
splitted_docs = RecursiveCharacterTextSplitter(chunk_size=1000,  chunk_overlap=200).split_documents(docs)
vectorStore = FAISS.from_documents(splitted_docs, HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))
retriever = vectorStore.as_retriever()

In [4]:
from langchain.tools.retriever import create_retriever_tool

# Create tool for the retriever
retriever_tool = create_retriever_tool(retriever, name='langchain-search', description='Search any information about langchain tools') 
print(retriever_tool.name)

langchain-search


In [5]:
# Combining the tools
tools = [wiki_tool, arxiv_tool, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Programs\\anaconda3\\envs\\langchain\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=1000)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=1000)),
 Tool(name='langchain-search', description='Search any information about langchain tools', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001EC5CC24A60>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000

## Running tools using Agents

In [6]:
from langchain_groq import ChatGroq
from langchain import hub

# Create model
llm = ChatGroq(model="Gemma2-9b-it")

# Get prompt from hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [7]:
# Creating Agent
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [10]:
# Creating Agent Executor
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [12]:
# Invoking Agent Executor
agent_executor.invoke({"input": "What are langchain tools?"})



> Entering new AgentExecutor chain...

Invoking: `langchain-search` with `{'query': 'What are langchain tools?'}`


Tools | ü¶úÔ∏èüîó LangChain

for doc...File SystemLangChain provides tools for interacting with a local file system out...FinancialDatasets ToolkitThe financial datasets stock market API provides REST endpoints that ...Github ToolkitThe Github toolkit contains tools that enable an LLM agent to interac...Gitlab ToolkitThe Gitlab toolkit contains tools that enable an LLM agent to interac...Gmail ToolkitThis will help you getting started with the GMail toolkit. This toolk...Golden QueryGolden provides a set of natural language APIs for querying and enric...Google BooksOverviewGoogle Cloud Text-to-SpeechGoogle Cloud Text-to-Speech enables developers to synthesize natural-...Google DriveThis notebook walks through connecting a LangChain to the Google Driv...Google FinanceThis notebook goes over how to use the Google Finance Tool to get inf...Google ImagenImagen on Vertex A

{'input': 'What are langchain tools?',
 'output': 'LangChain tools are utilities designed to be called by a model. Their inputs are designed to be generated by models, and their outputs are designed to be passed back to models. \n\nEssentially, they allow language models to interact with external systems and data sources. \n\n\n'}

In [14]:
agent_executor.invoke({"input": "What are multimodal models?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'multimodal models'}`


Page: Large language model
Summary: A large language model (LLM) is a type of machine learning model designed for natural language processing tasks such as language generation. LLMs are language models with many parameters, and are trained with self-supervised learning on a vast amount of text.
The largest and most capable LLMs are generative pretrained transformers (GPTs). Modern models can be fine-tuned for specific tasks or guided by prompt engineering. These models acquire predictive power regarding syntax, semantics, and ontologies inherent in human language corpora, but they also inherit inaccuracies and biases present in the data they are trained in.

That's a good start!  

Multimodal models go beyond just understanding and generating text. They're trained on **multiple types of data**, like text, images, audio, and even video. This allows them to understand and generate conte

{'input': 'What are multimodal models?',
 'output': "That's a good start!  \n\nMultimodal models go beyond just understanding and generating text. They're trained on **multiple types of data**, like text, images, audio, and even video. This allows them to understand and generate content in a more richer and complete way. \n\nThink of it like this: a unimodal model (like the large language model described) is like someone who only speaks one language. A multimodal model is like someone who speaks multiple languages and can also understand and interpret images, sounds, and even body language. \n\n\n\n"}

In [15]:
agent_executor.invoke({"input": "What is the title of paper 1706.03762"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks in an encoder-decoder configuration. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer, based
solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to be
superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014
English-to-German translation task, improving over the existing best results,
including ensembles by over 

{'input': 'What is the title of paper 1706.03762',
 'output': 'Title: Attention Is All You Need \n'}